In [24]:
import numpy as np
import pickle as pkl

import sys
from keras._tf_keras.keras.preprocessing import sequence

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report


from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Embedding, Bidirectional, LSTM, Dense, Dropout, Lambda, 
    Input, Concatenate, Reshape, Conv1D, Layer, Activation, 
    GlobalAveragePooling1D, GlobalMaxPooling1D, Multiply)
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import keras.src.legacy.backend as K


In [25]:
def result_evaluation(y_test, pred_test):

    pred_matrix = np.zeros_like(pred_test, dtype=np.int8)

    y_matrix = np.zeros_like(y_test, dtype=np.int8)
    pred_indexs = np.argmax(pred_test, 1)
    y_indexs = np.argmax(y_test, 1)

    for i in range(len(pred_indexs)):
        pred_matrix[i][pred_indexs[i]] = 1
        y_matrix[i][y_indexs[i]] = 1

    count_matrix = np.zeros((4, 3))
    for class_idx in range(4):

        count_matrix[class_idx][0] = np.sum(
            np.array(pred_matrix[:, class_idx]) * np.array(y_matrix[:, class_idx]))  # tp
        count_matrix[class_idx][1] = np.sum(np.array(
            pred_matrix[:, class_idx]) * (1 - np.array(y_matrix[:, class_idx])))  # fp
        count_matrix[class_idx][2] = np.sum(
            (1 - np.array(pred_matrix[:, class_idx])) * np.array(y_matrix[:, class_idx]))  # fn

    sumtp = sumfp = sumfn = 0

    for i in range(4):
        sumtp += count_matrix[i][0]
        sumfp += count_matrix[i][1]
        sumfn += count_matrix[i][2]

        precision = recall = f1 = 0

    if (sumtp + sumfp) == 0:
        precision = 0.
    else:
        precision = float(sumtp) / (sumtp + sumfp)

    if (sumtp + sumfn) == 0:
        recall = 0.
    else:
        recall = float(sumtp) / (sumtp + sumfn)

    if (precision + recall) == 0.:
        f1 = 0.
    else:
        f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1

In [26]:
# read train and test data which are pkl files

with open('train.pkl', 'rb') as f_Train:
    train_labels_vec = pkl.load(f_Train)
    train_word = pkl.load(f_Train)

    train_POS = pkl.load(f_Train)

    train_distances = pkl.load(f_Train)

    train_distances2 = pkl.load(f_Train)

    train_shortest_word = pkl.load(f_Train)

    train_shortest_pos = pkl.load(f_Train)
    train_shortest_dis1 = pkl.load(f_Train)
    train_shortest_dis2 = pkl.load(f_Train)

    train_entity = pkl.load(f_Train)



with open('test.pkl', 'rb') as f_Test:
    test_labels_vec = pkl.load(f_Test)
    test_word = pkl.load(f_Test)
    test_POS = pkl.load(f_Test)
    test_distances = pkl.load(f_Test)

    test_distances2 = pkl.load(f_Test)

    test_shortest_word = pkl.load(f_Test)

    test_shortest_pos = pkl.load(f_Test)
    test_shortest_dis1 = pkl.load(f_Test)
    test_shortest_dis2 = pkl.load(f_Test)

    test_entity = pkl.load(f_Test)


with open('vec.pkl', 'rb') as f_word2vec:
    vec_table = pkl.load(f_word2vec, encoding='latin1')
    pos_vec_table = pkl.load(f_word2vec, encoding='latin1')
    dis_vec_table = pkl.load(f_word2vec, encoding='latin1')


In [27]:
answer_y = np.array(test_labels_vec, dtype=np.int8)
train_y = np.array(train_labels_vec, dtype=np.int8)

In [28]:
s = {
    'emb_dimension': 100,  # dimension of word embedding
    'mini_batch': 64,
    'shortest_part_length': 12,
    'epochs': 60,
    'class_type': 5,
    'first_hidden_layer': 100,
    'lr': 0.001,
    'emb_dropout': 0.7,
    'dense_dropout': 0.5,
    'train_file': "../data/train.pkl.gz",
    'test_file': "../data/test.pkl.gz",
    'wordvecfile': "../data/vec.pkl.gz",

}
nb_class=s['class_type']
test_word_bak = test_word

word_dep_max = 0
word_max_num = 0
temp_max = 0

new_train_word = []
new_train_pos = []
new_train_dis = []
new_train_dis2 = []
new_test_word = []
new_test_pos = []
new_test_dis = []
new_test_dis2 = []

word_length = []

# calculate the max length of each subsequence
for i in range(3):

    temp_max = 0
    temp_train_word = []
    temp_train_pos = []
    temp_train_dis = []
    temp_train_dis2 = []
    for j in range(len(train_word)):

        assert len(train_word[j][i]) == len(train_POS[j][i])
        assert len(train_POS[j][i]) == len(train_distances[j][i])
        assert len(train_distances[j][i]) == len(train_distances2[j][i])
        temp_train_word.append(train_word[j][i])
        temp_train_pos.append(train_POS[j][i])
        temp_train_dis.append(train_distances[j][i])
        temp_train_dis2.append(train_distances2[j][i])
        if len(train_word[j][i]) > temp_max:
            temp_max = len(train_word[j][i])

    new_train_word.append(temp_train_word)
    new_train_pos.append(temp_train_pos)
    new_train_dis.append(temp_train_dis)
    new_train_dis2.append(temp_train_dis2)

    temp_test_word = []
    temp_test_pos = []
    temp_test_dis = []
    temp_test_dis2 = []
    for j in range(len(test_word)):
        assert len(test_word[j][i]) == len(test_POS[j][i])
        assert len(test_POS[j][i]) == len(test_distances[j][i])
        assert len(test_distances[j][i]) == len(test_distances2[j][i])
        temp_test_word.append(test_word[j][i])
        temp_test_pos.append(test_POS[j][i])
        temp_test_dis.append(test_distances[j][i])
        temp_test_dis2.append(test_distances2[j][i])
        if len(test_word[j][i]) > temp_max:
            temp_max = len(test_word[j][i])

    new_test_word.append(temp_test_word)
    new_test_pos.append(temp_test_pos)
    new_test_dis.append(temp_test_dis)
    new_test_dis2.append(temp_test_dis2)
    word_length.append(temp_max)

train_word = new_train_word

train_POS = new_train_pos
train_distances = new_train_dis
train_distances2 = new_train_dis2

test_word = new_test_word

test_POS = new_test_pos
test_distances = new_test_dis
test_distances2 = new_test_dis2

train_entity_0 = []
train_entity_1 = []
test_entity_0 = []
test_entity_1 = []
for i in range(len(train_entity)):
    temp_list = []
    temp_list.append(train_entity[i][0])
    train_entity_0.append(temp_list)
    temp_list = []
    temp_list.append(train_entity[i][1])
    train_entity_1.append(temp_list)
for i in range(len(test_entity)):
    temp_list = []
    temp_list.append(test_entity[i][0])
    test_entity_0.append(temp_list)
    temp_list = []
    temp_list.append(test_entity[i][1])
    test_entity_1.append(temp_list)

print('Pad sequences (samples x time)')

train_word_list = []
test_word_list = []
train_POS_list = []
test_POS_list = []

train_distances_list = []
test_distances_list = []
train_distances2_list = []
test_distances2_list = []

for i in range(len(train_word)):
    train_word_list.append(
        sequence.pad_sequences(train_word[i], maxlen=word_length[i], truncating='post', padding='post'))
for i in range(len(test_word)):
    test_word_list.append(sequence.pad_sequences(test_word[i], maxlen=word_length[i], truncating='post',
                                                 padding='post'))

for i in range(len(train_POS)):
    train_POS_list.append(
        sequence.pad_sequences(train_POS[i], maxlen=word_length[i], truncating='post', padding='post'))
for i in range(len(test_distances)):
    test_POS_list.append(sequence.pad_sequences(test_POS[i], maxlen=word_length[i], truncating='post',
                                                padding='post'))

for i in range(len(train_distances)):
    train_distances_list.append(
        sequence.pad_sequences(train_distances[i], maxlen=word_length[i], truncating='post', padding='post'))
for i in range(len(test_distances)):
    test_distances_list.append(sequence.pad_sequences(test_distances[i], maxlen=word_length[i], truncating='post',
                                                      padding='post'))

for i in range(len(train_distances2)):
    train_distances2_list.append(
        sequence.pad_sequences(train_distances2[i], maxlen=word_length[i], truncating='post', padding='post'))
for i in range(len(test_distances2)):
    test_distances2_list.append(sequence.pad_sequences(test_distances2[i], maxlen=word_length[i], truncating='post',
                                                       padding='post'))

train_shortest_word = sequence.pad_sequences(
    train_shortest_word, maxlen=s['shortest_part_length'], truncating='post', padding='post')
test_shortest_word = sequence.pad_sequences(test_shortest_word, maxlen=s['shortest_part_length'], truncating='post',
                                            padding='post')

train_shortest_pos = sequence.pad_sequences(
    train_shortest_pos, maxlen=s['shortest_part_length'], truncating='post', padding='post')
test_shortest_pos = sequence.pad_sequences(
    test_shortest_pos, maxlen=s['shortest_part_length'], truncating='post', padding='post')
train_shortest_dis1 = sequence.pad_sequences(
    train_shortest_dis1, maxlen=s['shortest_part_length'], truncating='post', padding='post')
test_shortest_dis1 = sequence.pad_sequences(
    test_shortest_dis1, maxlen=s['shortest_part_length'], truncating='post', padding='post')
train_shortest_dis2 = sequence.pad_sequences(
    train_shortest_dis2, maxlen=s['shortest_part_length'], truncating='post', padding='post')
test_shortest_dis2 = sequence.pad_sequences(
    test_shortest_dis2, maxlen=s['shortest_part_length'], truncating='post', padding='post')

train_entity_0 = sequence.pad_sequences(
    train_entity_0, maxlen=1, truncating='post', padding='post')
train_entity_1 = sequence.pad_sequences(
    train_entity_1, maxlen=1, truncating='post', padding='post')
test_entity_0 = sequence.pad_sequences(
    test_entity_0, maxlen=1, truncating='post', padding='post')
test_entity_1 = sequence.pad_sequences(
    test_entity_1, maxlen=1, truncating='post', padding='post')

print(train_shortest_word.shape)


# embedding layer

wordembedding = Embedding(vec_table.shape[0],
                          vec_table.shape[1],
                          weights=[vec_table])

disembedding = Embedding(dis_vec_table.shape[0],
                         dis_vec_table.shape[1],
                         weights=[dis_vec_table]
                         )

posembedding = Embedding(pos_vec_table.shape[0],
                         pos_vec_table.shape[1],
                         weights=[pos_vec_table]
                         )

input_entity_0 = Input(shape=(1,), dtype='int32', name='input_entity_0')
entity_fea_0 = wordembedding(input_entity_0)
input_entity_1 = Input(shape=(1,), dtype='int32', name='input_entity_1')
entity_fea_1 = wordembedding(input_entity_1)

input_word_0 = Input(
    shape=(word_length[0],), dtype='int32', name='input_word_0')
word_fea_0 = wordembedding(input_word_0)  # trainable=False

input_pos_0 = Input(
    shape=(word_length[0],), dtype='int32', name='input_pos_0')
pos_fea_0 = posembedding(input_pos_0)

input_dis1_0 = Input(
    shape=(word_length[0],), dtype='int32', name='input_dis1_0')
dis_fea1_0 = disembedding(input_dis1_0)

input_dis2_0 = Input(
    shape=(word_length[0],), dtype='int32', name='input_dis2_0')
dis_fea2_0 = disembedding(input_dis2_0)

input_word_1 = Input(
    shape=(word_length[1],), dtype='int32', name='input_word_1')
word_fea_1 = wordembedding(input_word_1)

input_pos_1 = Input(
    shape=(word_length[1],), dtype='int32', name='input_pos_1')
pos_fea_1 = posembedding(input_pos_1)

input_dis1_1 = Input(
    shape=(word_length[1],), dtype='int32', name='input_dis1_1')
dis_fea1_1 = disembedding(input_dis1_1)

input_dis2_1 = Input(
    shape=(word_length[1],), dtype='int32', name='input_dis2_1')
dis_fea2_1 = disembedding(input_dis2_1)

input_word_2 = Input(
    shape=(word_length[2],), dtype='int32', name='input_word_2')
word_fea_2 = wordembedding(input_word_2)

input_pos_2 = Input(
    shape=(word_length[2],), dtype='int32', name='input_pos_2')
pos_fea_2 = posembedding(input_pos_2)

input_dis1_2 = Input(
    shape=(word_length[2],), dtype='int32', name='input_dis1_2')
dis_fea1_2 = disembedding(input_dis1_2)

input_dis2_2 = Input(
    shape=(word_length[2],), dtype='int32', name='input_dis2_2')
dis_fea2_2 = disembedding(input_dis2_2)

emb_merge_0 = Concatenate()(
    [word_fea_0, pos_fea_0, dis_fea1_0, dis_fea2_0])

emb_merge_1 = Concatenate()(
    [word_fea_1, pos_fea_1, dis_fea1_1, dis_fea2_1])
emb_merge_2 = Concatenate()(
    [word_fea_2, pos_fea_2, dis_fea1_2, dis_fea2_2])

input_shortest_word = Input(
    shape=(s['shortest_part_length'],), dtype='int32', name='input_shortest_word')
shortest_word_fea = wordembedding(input_shortest_word)

input_shortest_pos = Input(
    shape=(s['shortest_part_length'],), dtype='int32', name='input_shortest_pos')

shortest_pos_fea = posembedding(input_shortest_pos)

shortest_input_dis1 = Input(
    shape=(s['shortest_part_length'],), dtype='int32', name='shortest_input_dis1')
shortest_dis_fea1 = disembedding(shortest_input_dis1)

shortest_input_dis2 = Input(
    shape=(s['shortest_part_length'],), dtype='int32', name='shortest_input_dis2')
shortest_dis_fea2 = disembedding(shortest_input_dis2)

emb_merge_shortest = Concatenate()(
    [shortest_word_fea, shortest_pos_fea, shortest_dis_fea1, shortest_dis_fea2])


Pad sequences (samples x time)
(27792, 12)


In [ ]:
class emb_AttentionLayer(Layer):
    def __init__(self, **kwargs):

        super(emb_AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        print(input_shape)
        self.scaler=input_shape[0][1]
        print(self.scaler)

        super(emb_AttentionLayer, self).build(input_shape)

    def call(self, x, mask=None):

        sentence = x[0]
        entity_0=x[1]
        entity_1=x[2]

        eij_0=K.batch_dot(sentence,entity_0,axes=[2,2])
        K.sqrt(sentence)
        K.sqrt(entity_0)
        eij_0/=2000
        # print('eij', eij_0.shape, x[0].shape[0],x[0].shape[1], tf.reshape(eij_0, (-1, self.scaler)))
        eij_0 = tf.reshape(eij_0, (-1, self.scaler))
        ai_0 = K.exp(eij_0)
        weights_0=ai_0 / tf.expand_dims(K.sum(ai_0, axis=1), axis=-1)

        eij_1 = K.batch_dot(sentence, entity_1, axes=[2, 2])
        eij_1 /= 2000
        eij_1 = tf.reshape(eij_1, (-1, self.scaler))
        ai_1 = K.exp(eij_1)
        weights_1 = ai_1 / tf.expand_dims(K.sum(ai_1, axis=1), axis=-1)


        weights=((weights_0+weights_1)/2.0)*self.scaler

        weighted_input = x[3] * tf.expand_dims(weights, axis=-1)

        return weighted_input

    def get_output_shape_for(self, input_shape):
        return (input_shape[3])


# attention layer
att_emb_merge_0 = emb_AttentionLayer()(
    [word_fea_0, entity_fea_0, entity_fea_1, emb_merge_0])
att_emb_merge_1 = emb_AttentionLayer()(
    [word_fea_1, entity_fea_0, entity_fea_1, emb_merge_1])
att_emb_merge_2 = emb_AttentionLayer()(
    [word_fea_2, entity_fea_0, entity_fea_1, emb_merge_2])
att_emb_merge_shortest = emb_AttentionLayer()(
    [shortest_word_fea, entity_fea_0, entity_fea_1, emb_merge_shortest])

[(None, 60, 200), (None, 1, 200), (None, 1, 200), (None, 60, 230)]
60
[(None, 60, 200), (None, 1, 200), (None, 1, 200), (None, 60, 230)]
60
[(None, 60, 200), (None, 1, 200), (None, 1, 200), (None, 60, 230)]
60
[(None, 12, 200), (None, 1, 200), (None, 1, 200), (None, 12, 230)]
12


In [30]:
# class CBAM_EntityAttentionLayer(Layer):
#     def __init__(self, **kwargs):
#         super(CBAM_EntityAttentionLayer, self).__init__(**kwargs)
#         self.shared_dense_one = None
#         self.shared_dense_two = None
#         self.spatial_conv = None

#     def build(self, input_shape):
#         self.scaler = input_shape[0][1]
#         channel_shape = input_shape[3][-1]

#         self.shared_dense_one = Dense(units=channel_shape // 8, activation='relu')
#         self.shared_dense_two = Dense(units=channel_shape, activation='sigmoid')
#         self.spatial_conv = Conv1D(filters=1, kernel_size=1, activation='sigmoid')

#         super(CBAM_EntityAttentionLayer, self).build(input_shape)

#     def call(self, inputs, mask=None):
#         sentence, entity_, entity_1, merged_features = inputs

#         # Calculate entity distance-based attention weights
#         eij_ = K.batch_dot(sentence, entity_, axes=[2, 2]) / 200
#         eij_ = tf.reshape(eij_, (-1, self.scaler))
#         ai_ = K.exp(eij_)
#         weights_ = ai_ / tf.expand_dims(K.sum(ai_, axis=1), axis=-1)

#         eij_1 = K.batch_dot(sentence, entity_1, axes=[2, 2]) / 200
#         eij_1 = tf.reshape(eij_1, (-1, self.scaler))
#         ai_1 = K.exp(eij_1)
#         weights_1 = ai_1 / tf.expand_dims(K.sum(ai_1, axis=1), axis=-1)

#         weights = ((weights_ + weights_1) / 2.) * self.scaler

#         # Apply channel attention
#         avg_pool = GlobalAveragePooling1D()(merged_features)
#         max_pool = GlobalMaxPooling1D()(merged_features)

#         avg_attention = self.shared_dense_two(self.shared_dense_one(avg_pool))
#         max_attention = self.shared_dense_two(self.shared_dense_one(max_pool))
        
#         channel_attention = Multiply()([merged_features, tf.expand_dims(avg_attention + max_attention, axis=1)])

#         # Apply spatial attention
#         avg_pool_spatial = tf.reduce_mean(channel_attention, axis=-1, keepdims=True)
#         max_pool_spatial = tf.reduce_max(channel_attention, axis=-1, keepdims=True)
#         concat = tf.concat([avg_pool_spatial, max_pool_spatial], axis=-1)
        
#         spatial_attention = self.spatial_conv(concat)
#         spatial_attention = Multiply()([spatial_attention, tf.expand_dims(weights, -1)])

#         weighted_input = Multiply()([channel_attention, spatial_attention])

#         return weighted_input

#     def compute_output_shape(self, input_shape):
#         return input_shape[3]


# # attention layer
# att_emb_merge_0 = CBAM_EntityAttentionLayer()(
#     [word_fea_0, entity_fea_0, entity_fea_1, emb_merge_0])
# att_emb_merge_1 = CBAM_EntityAttentionLayer()(
#     [word_fea_1, entity_fea_0, entity_fea_1, emb_merge_1])
# att_emb_merge_2 = CBAM_EntityAttentionLayer()(
#     [word_fea_2, entity_fea_0, entity_fea_1, emb_merge_2])
# att_emb_merge_shortest = CBAM_EntityAttentionLayer()(
#     [shortest_word_fea, entity_fea_0, entity_fea_1, emb_merge_shortest])

In [ ]:
# def cbam_attention_layer(inputs):
#     word_features, entity_features_, entity_features_1, merged_features = inputs
    
#     # Example Channel Attention applied on word features
#     channel_avg_pool = GlobalAveragePooling1D()(merged_features)
#     channel_max_pool = GlobalMaxPooling1D()(merged_features)
    
#     shared_dense_one = Dense(units=channel_avg_pool.shape[-1] // 8, activation='relu')
#     shared_dense_two = Dense(units=channel_avg_pool.shape[-1], activation='sigmoid')

#     avg_attention_scores = shared_dense_two(shared_dense_one(channel_avg_pool))
#     max_attention_scores = shared_dense_two(shared_dense_one(channel_max_pool))

#     channel_attention_scores = Add()([avg_attention_scores, max_attention_scores])
    
#     # Channel Attention
#     channel_attention_scores = Lambda(lambda x: x[:, None, :])(channel_attention_scores)  # Add a dimension
#     channel_attention_output = Multiply()([merged_features, channel_attention_scores])
    
#     # Example Spatial Attention
#     avg_pool = Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(channel_attention_output)
#     max_pool = Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(channel_attention_output)
#     concat = Concatenate(axis=-1)([avg_pool, max_pool])
    
#     spatial_attention_scores = tf.keras.layers.Conv1D(filters=1, kernel_size=1, activation='sigmoid')(concat)
#     spatial_attention_output = Multiply()([channel_attention_output, spatial_attention_scores])
    
#     return spatial_attention_output

# # attention layer
# att_emb_merge_0 = cbam_attention_layer(
#     [word_fea_0, entity_fea_0, entity_fea_1, emb_merge_0])
# att_emb_merge_1 = cbam_attention_layer(
#     [word_fea_1, entity_fea_0, entity_fea_1, emb_merge_1])
# att_emb_merge_2 = cbam_attention_layer(
#     [word_fea_2, entity_fea_0, entity_fea_1, emb_merge_2])
# att_emb_merge_shortest = cbam_attention_layer(
#     [shortest_word_fea, entity_fea_0, entity_fea_1, emb_merge_shortest])

In [32]:
# dropout layer
emb_merge_0 = Dropout(s['emb_dropout'])(att_emb_merge_0)
emb_merge_1 = Dropout(s['emb_dropout'])(att_emb_merge_1)
emb_merge_2 = Dropout(s['emb_dropout'])(att_emb_merge_2)
emb_merge_shortest = Dropout(s['emb_dropout'])(att_emb_merge_shortest)
# LSTM Layers
gru_merge_0 = Bidirectional(LSTM(units=100))(emb_merge_0)
gru_merge_1 = Bidirectional(LSTM(units=100))(emb_merge_1)
gru_merge_2 = Bidirectional(LSTM(units=100))(emb_merge_2)
gru_merge_shortest = Bidirectional(LSTM(units=100))(emb_merge_shortest)

# lstm_out = Bidirectional(LSTM(units=100))(emb_merge_0)

In [33]:
gru_merge_0 = Reshape((1, 2*s['first_hidden_layer']))(gru_merge_0)
gru_merge_1 = Reshape((1, 2*s['first_hidden_layer']))(gru_merge_1)
gru_merge_2 = Reshape((1, 2*s['first_hidden_layer']))(gru_merge_2)
gru_merge_shortest = Reshape(
    (1, 2*s['first_hidden_layer']))(gru_merge_shortest)

In [34]:
gru_merge_temp = merged = Concatenate()([gru_merge_0, entity_fea_0,gru_merge_1,gru_merge_shortest,entity_fea_1,gru_merge_2])

In [35]:
gru_merge_top = Bidirectional(LSTM(units=100))(gru_merge_temp)

In [36]:
dense_drop_0=Dropout(s['dense_dropout'])(gru_merge_top)

final_output=Dense(nb_class, activation='softmax')(dense_drop_0)

In [37]:
model = Model([input_word_0,input_word_1,input_word_2,input_pos_0,input_pos_1,input_pos_2,input_dis1_0,input_dis1_1,input_dis1_2,
                         input_dis2_0,input_dis2_1,input_dis2_2,input_shortest_word, shortest_input_dis1,shortest_input_dis2,
                         input_shortest_pos,input_entity_0,input_entity_1], final_output)

In [38]:
def compile_model(model):
    model.compile(optimizer=RMSprop(learning_rate=.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [39]:
compile_model(model)

In [17]:
model.fit(x=[train_word_list[0], train_word_list[1], train_word_list[2], train_POS_list[0], train_POS_list[1], train_POS_list[2],
                          train_distances_list[0], train_distances_list[1], train_distances_list[2], train_distances2_list[0],
                          train_distances2_list[1], train_distances2_list[2], train_shortest_word, train_shortest_dis1, 
                           train_shortest_dis2, train_shortest_pos, train_entity_0, train_entity_1], y=train_y, batch_size=64, shuffle=True, epochs=60) 

Epoch 1/60


/Users/amir/Programming/University/NLP_Class/venv/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_word_0', 'input_word_1', 'input_word_2', 'input_pos_0', 'input_pos_1', 'input_pos_2', 'input_dis1_0', 'input_dis1_1', 'input_dis1_2', 'input_dis2_0', 'input_dis2_1', 'input_dis2_2', 'input_shortest_word', 'shortest_input_dis1', 'shortest_input_dis2', 'input_shortest_pos', 'input_entity_0', 'input_entity_1']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*')
  warnings.warn(


435/435 ━━━━━━━━━━━━━━━━━━━━ 37s 75ms/step - accuracy: 0.8426 - loss: 0.5770
Epoch 2/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 33s 77ms/step - accuracy: 0.8548 - loss: 0.5319
Epoch 3/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 33s 76ms/step - accuracy: 0.8533 - loss: 0.5020
Epoch 4/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 34s 77ms/step - accuracy: 0.8544 - loss: 0.4812
Epoch 5/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 33s 77ms/step - accuracy: 0.8560 - loss: 0.4788
Epoch 6/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 34s 78ms/step - accuracy: 0.8582 - loss: 0.4717
Epoch 7/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 33s 76ms/step - accuracy: 0.8577 - loss: 0.4627
Epoch 8/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 32s 75ms/step - accuracy: 0.8601 - loss: 0.4504
Epoch 9/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 33s 77ms/step - accuracy: 0.8609 - loss: 0.4559
Epoch 10/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 34s 79ms/step - accuracy: 0.8583 - loss: 0.4528
Epoch 11/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 35s 79ms/step - accuracy: 0.8639 - loss: 0.4390
Epoch 12/60
435/435 ━━━━━━━━━━━━━━━━━━━━ 

In [21]:
pred_test = model.predict([test_word_list[0], test_word_list[1], test_word_list[2], test_POS_list[0], test_POS_list[1], test_POS_list[2],
                          test_distances_list[0], test_distances_list[1], test_distances_list[2], test_distances2_list[0],
                          test_distances2_list[1], test_distances2_list[2], test_shortest_word, test_shortest_dis1, test_shortest_dis2, test_shortest_pos, test_entity_0, test_entity_1],
                          batch_size=s['mini_batch'])  # ,test_POS

precision, recall, F1 = result_evaluation(answer_y, pred_test)
# print(testing epochs:' + ' precision:' + str(np.round(precision, 5)) + ' recall:' + str(np.round(recall, 5)) + ' F1:' + str(np.round(F1, 5))

print('testing epochs:' + str(60)+' precision:' + str(np.round(precision, 5)
                                                         ) + ' recall:' + str(np.round(recall, 5)) + ' F1:' + str(np.round(F1, 5)))

84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step
testing epochs:60 precision:0.65134 recall:0.79571 F1:0.71632


In [23]:
# model.save('my_model.keras')

In [40]:
model.load_weights("my_model.h5")

In [41]:
pred_test = model.predict([test_word_list[0], test_word_list[1], test_word_list[2], test_POS_list[0], test_POS_list[1], test_POS_list[2],
                          test_distances_list[0], test_distances_list[1], test_distances_list[2], test_distances2_list[0],
                          test_distances2_list[1], test_distances2_list[2], test_shortest_word, test_shortest_dis1, test_shortest_dis2, test_shortest_pos, test_entity_0, test_entity_1],
                          batch_size=s['mini_batch'])

/Users/amir/Programming/University/NLP_Class/venv/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_word_0', 'input_word_1', 'input_word_2', 'input_pos_0', 'input_pos_1', 'input_pos_2', 'input_dis1_0', 'input_dis1_1', 'input_dis1_2', 'input_dis2_0', 'input_dis2_1', 'input_dis2_2', 'input_shortest_word', 'shortest_input_dis1', 'shortest_input_dis2', 'input_shortest_pos', 'input_entity_0', 'input_entity_1']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*')
  warnings.warn(


84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step


In [73]:
# # np.argmax(pred_test.round(2), 1)
# pred_test.round(2)
# y_test = np.zeros_like(pred_test)
# max_indices = np.argmax(pred_test, axis=1)
# y_test[np.arange(pred_test.shape[0]), max_indices] = 1

In [42]:
answer_y = np.argmax(answer_y, axis=1)
pred_test = np.argmax(pred_test, axis=1)

# precision = precision_score(answer_y, y_test)#, average='weighted')
# recall = recall_score(answer_y, y_test)#, average='weighted')
# f1 = f1_score(answer_y, y_test)#, average='weighted')

# Print the results
# print(f'Precision: {precision:.2f}')
# print(f'Recall: {recall:.2f}')
# print(f'F1-Score: {f1:.2f}')
print(classification_report(answer_y, pred_test))

              precision    recall  f1-score   support

           0       0.61      0.86      0.71       360
           1       0.62      0.80      0.70       302
           2       0.79      0.85      0.82       221
           3       0.71      0.43      0.53        96
           4       0.97      0.92      0.95      4390

    accuracy                           0.90      5369
   macro avg       0.74      0.77      0.74      5369
weighted avg       0.91      0.90      0.90      5369

